In [285]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import urllib.request
import re
from datetime import datetime
import warnings
import logging
import pickle
import warnings
import pickle
import argparse
import os
from collections import defaultdict
from underthesea import pos_tag, chunk, sentiment
from core_nlp.tokenization import base_tokenizer, dict_models

# https://www.dataquest.io/blog/python-json-tutorial

# LINK = 'http://10.220.92.51:8983/solr/master_adayroi_PriceRow_default/select?fl=productNameForSearch_text_vi,code_string,brandName_text_vi_mv&fq=(catalogId:%22adayroiProductCatalog%22%20AND%20catalogVersion:%22Online%22)&fq=isValidPrice_boolean:true&fq=isonline_boolean:true&indent=on&q=*:*&wt=json'
# LINK = 'http://solrslave01.adayroi.com/solr/master_adayroi_PriceRow_default/select?fl=code_string,categoryNameForSearch_text_vi_mv,categoryName_text_vi_mv,description_text_vi,productNameForSearch_text_vi,brandName_text_vi_mv&fq=(catalogId:%22adayroiProductCatalog%22%20AND%20catalogVersion:%22Online%22)&fq=isValidPrice_boolean:true&fq=isonline_boolean:true&indent=on&q=*:*&wt=json'

# warnings.simplefilter('ignore')
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s', level = logging.INFO)
# LINK = 'http://10.220.92.51:8983/solr/master_adayroi_PriceRow_default/select?fl=productNameForSearch_text_vi,code_string,brandName_text_vi_mv&fq=(catalogId:%22adayroiProductCatalog%22%20AND%20catalogVersion:%22Online%22)&fq=isValidPrice_boolean:true&fq=isonline_boolean:true&indent=on&q=*:*&wt=json'

with open('config.json', 'rb') as fp:
    config = json.loads(fp.read())
print('config sentiments: ', config['link_sentiment'])

parser = argparse.ArgumentParser(description='Tagging Adayroi onsite production')
parser.add_argument('--link', default=config['link'], help='link to clone data from solr')
parser.add_argument('--num_product', default=config['num_product'], type=int, help='number of product')
parser.add_argument('--link_old', default=config['link_old'], help='link tagging product previous day')
parser.add_argument('--link_new', default=config['link_new'], help='link tagging product today')
parser.add_argument('--link_output', default=config['link_output'], help='link tagging product with two information productCode and tag_list')
parser.add_argument('--link_sentiment', default=config['link_sentiment'], help='link json file include good sentiment code')
parser.add_argument('--link_cate', default=config['link_cate'], help='link json file include good sentiment code')
argparser = vars(parser.parse_args([]))

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import urllib.request
import re
from datetime import datetime
import warnings
import logging
import pickle
import warnings
import pickle
import argparse
import os
from collections import defaultdict
from underthesea import pos_tag, chunk, sentiment
from core_nlp.tokenization import base_tokenizer, dict_models

# https://www.dataquest.io/blog/python-json-tutorial

# LINK = 'http://10.220.92.51:8983/solr/master_adayroi_PriceRow_default/select?fl=productNameForSearch_text_vi,code_string,brandName_text_vi_mv&fq=(catalogId:%22adayroiProductCatalog%22%20AND%20catalogVersion:%22Online%22)&fq=isValidPrice_boolean:true&fq=isonline_boolean:true&indent=on&q=*:*&wt=json'
# LINK = 'http://solrslave01.adayroi.com/solr/master_adayroi_PriceRow_default/select?fl=code_string,categoryNameForSearch_text_vi_mv,categoryName_text_vi_mv,description_text_vi,productNameForSearch_text_vi,brandName_text_vi_mv&fq=(catalogId:%22adayroiProductCatalog%22%20AND%20catalogVersion:%22Online%22)&fq=isValidPrice_boolean:true&fq=isonline_boolean:true&indent=on&q=*:*&wt=json'

# warnings.simplefilter('ignore')
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s', level = logging.INFO)
# LINK = 'http://10.220.92.51:8983/solr/master_adayroi_PriceRow_default/select?fl=productNameForSearch_text_vi,code_string,brandName_text_vi_mv&fq=(catalogId:%22adayroiProductCatalog%22%20AND%20catalogVersion:%22Online%22)&fq=isValidPrice_boolean:true&fq=isonline_boolean:true&indent=on&q=*:*&wt=json'

with open('config.json', 'rb') as fp:
    config = json.loads(fp.read())
print('config sentiments: ', config['link_sentiment'])

parser = argparse.ArgumentParser(description='Tagging Adayroi onsite production')
parser.add_argument('--link', default=config['link'], help='link to clone data from solr')
parser.add_argument('--num_product', default=config['num_product'], type=int, help='number of product')
parser.add_argument('--link_old', default=config['link_old'], help='link tagging product previous day')
parser.add_argument('--link_new', default=config['link_new'], help='link tagging product today')
parser.add_argument('--link_output', default=config['link_output'], help='link tagging product with two information productCode and tag_list')
parser.add_argument('--link_sentiment', default=config['link_sentiment'], help='link json file include good sentiment code')
parser.add_argument('--link_cate', default=config['link_cate'], help='link json file include good sentiment code')
args = vars(parser.parse_args([]))


class IOObject(object):
    def __init__(self, filename):
        self.filename = filename
        
    def _save_pickle(self, obj):
        with open(self.filename, "wb") as fp:   #Pickling
            pickle.dump(obj, fp)

    def _open_pickle(self):
        with open(self.filename, "rb") as fp:   #Unpickling
            obj = pickle.load(fp)
        return obj

    def _save_json(self, obj):
        with open(self.filename, "wb") as fp:
            json.dump(obj, fp)

    def _open_json(self):
        with open(self.filename, "rb") as fp:
            obj = json.loads(fp.read())
        return obj


class TagProductAll(object):
    def __init__(self, link):
        self.link = link
        self.dictionary = self._read_solr()
        self.sentiments = IOObject(args.link_sentiment)._open_json()

    # Đọc dữ liệu từ solr
    def _read_solr(self):
        response1 = urllib.request.urlopen(self.link)
        # dictionary1 = json.loads(response1.read())
        # no_product = dictionary1['response']['numFound']
        no_product = args.num_product
        logging.info('number of product get: {}'.format(no_product))
        self.link = self.link + '&rows=' + str(no_product)
        response2 = urllib.request.urlopen(self.link)
        dictionary2 = json.loads(response2.read())
        self.cate_map_level = self._read_cate(args.link_cate)
        logging.info('completed read link!')
        return dictionary2
    
    # Đọc các mô tả tốt
    def _read_sentiments(self):
        sentiments = json.loads(args.link_sentiments)
        return sentiments

    # Đọc dữ liệu cateName
    def _read_cate(self, cate_path=args.link_cate):
        datacate = pd.read_excel(cate_path, sheet_name='displaycate', header=0)
        cate_map_level = dict()
        for i, row in datacate[['LVL', 'CATENAME']].iterrows():
            cate_map_level[row[1]] = row[0]
        cate_map_level['Danh mục hiển thị'] = 0
        return cate_map_level

    # Lọc ra những cate cấp từ 3-4
    def _filter_cate(self, texts):
        return [text for text in texts if self.cate_map_level[text] in (3, 4)]

    # Lấy cate cuối cùng trong danh sách list các cate
    def _last_child_cate(self, texts):
        return sorted([(self.cate_map_level[text], text) for text in texts], key=lambda x: x[0], reverse=True)[0][1]

    # Tạo bảng dữ liệu gồm productCode, productName, tags
    def _create_dataset(self):
        productName = [p['productNameForSearch_text_vi'] for p in self.dictionary['response']['docs']]
        productCode = [p['code_string'] for p in self.dictionary['response']['docs']]
        cateName = []
        for p in self.dictionary['response']['docs']:
            try:
                cateName.append(self._last_child_cate(p['categoryName_text_vi_mv']))
            except:
                cateName.append('')

        branchName = []
        for p in self.dictionary['response']['docs']:
            try:
                branchName.append(p['brandName_text_vi_mv'][0])
            except:
                branchName.append('')

        description = []
        for p in self.dictionary['response']['docs']:
            try:
                description.append(p['description_text_vi'])
            except:
                description.append('')

        # drop duplicate productCode
        if os.path.exists(args.link_old):
            #Đọc file cũ
            dataset_old = pd.read_csv(args.link_old)
            #Đọc file mới
            dataset_update = pd.DataFrame({'productCode': productCode, 'productName': productName, 
                                'cateName': cateName, 'branchName': branchName,
                                'description': description})
            dataset_update.drop_duplicates(subset=['productCode'], inplace=True)
            #Danh sách các sản phẩm chung
            productCodeCommon = pd.merge(dataset_old, dataset_update, on = 'productCode')['productCode'].tolist()
            #Lấy những sản phẩm mới
            dataset = dataset_update[~dataset_update['productCode'].isin(productCodeCommon)]
            dataset.set_index(np.arange(dataset.shape[0]))
            logging.info('Dataset shape: {}'.format(dataset.shape))
        else:
            dataset = pd.DataFrame({'productCode': productCode, 'productName': productName, 
                                'cateName': cateName, 'branchName': branchName,
                                'description': description})
            dataset.drop_duplicates(subset=['productCode'], inplace=True)
            dataset.set_index(np.arange(dataset.shape[0]))
            logging.info('Dataset shape: {}'.format(dataset.shape))

        dataset = self._update_sentiments(dataset)
        # Cập nhật tag 1
        dataset['tag1'] = [self._gen_tag1(item) for k, item in dataset[['cateName', 'sentiments']].iterrows()]
        # Cập nhật tag 2
        dataset = self._gen_tag2(dataset)
        # cập nhật tag 3
        dataset['tag3'] = self._gen_tag3(dataset)
        # cập nhật tag 4
        dataset['tag4'] = self._gen_tag4(dataset)
        # combine các tags
        dataset = self._combine_tags(dataset)
        logging.info('Completed tag_list!')
        return dataset

    # Tạo từ điển mô tả các vị trí của từ trong câu
    # Ví dụ: 'Trời ơi thật là một ngày đẹp trời. Kết quả trả về bao gồm {'trời': [0, 7], 'ơi':[1], 'thật':[2], 'là':[3], 'một':[4], 'ngày':[5], 'đẹp':[6]}
    def _gen_dict_desc(self, text):
        desc_dict = defaultdict()
        for k, item in enumerate(text.split()):
            if item in desc_dict:
                desc_dict[item].append(k)
            else:
                desc_dict[item] = [k]
        return desc_dict

    # Tính toán khoảng cách nhỏ nhất của 2 cụm vị trí
    # Ví dụ: 'trời xanh mây xanh' sau khi tạo từ điển vị trí sẽ là {'trời':[0], 'xanh':[1, 3], 'mây': [2]}
    # Đo khoảng cách giữa từ 'xanh' và 'trời' sẽ là 1. Mục đích là để tìm các từ có thể ghép cặp với nhau trong 1 khoảng cách nhất định nếu việc tìm từ đó liền kề nhau là không thể.
    # VD: 'trong và sạch' có thể được ghép cặp coi như từ 'trong sạch' bằng cách so khớp khoảng cách.
    def _cross_distance(self, first_list, second_list):
        return np.min([(y - x) for x in first_list for y in second_list if (y - x) >= 0])

    def _check_collocation(self, first, last, desc_dict, min_distance=2):
        try:
            distance = self._cross_distance(desc_dict[first], desc_dict[last])
        except:
            return False
        if (distance <= min_distance) & (distance >= 0):
            return True
        else:
            return False

    # print(_check_collocation('cho', 'tươi', desc_dict = desc_dict))

    # Loại bỏ các kí tự đặc biệt bằng khoảng trắng
    def _clean_text(self, text, regex=r'[-\(\)\"#/@;:<>{}\[\]`+=~%$&|.*!?,|\u200b]'):
        text = re.sub(regex, r' ', text.lower())
        text = re.sub(r'\s+', r' ', text)
        return text

    # Kiểm tra một cụm từ có trong mô tả hay không với khoảng cách là 2
    def _check_desc(self, search, description, min_distance=2):
        words = search.split()
        description = self._clean_text(description)
        desc_dict = self._gen_dict_desc(description)
        if len(words) == 1:
            if not words[0] in desc_dict:
                return False
        else:
            for i in range(len(words) - 1):
                if not self._check_collocation(words[i], words[i + 1], desc_dict, min_distance):
                    return False
        return True

    # Lấy ra các comment tốt
    def _extract_good_sents(self, text):
        return list(set([item for item in self.sentiments if self._check_desc(item.lower(), text.lower())]))


    # _check_desc('internet', dataset['description'][0])

    # Cập nhật trường comment vào dataset
    def _update_sentiments(self, dataset):
        sentiments_prod = []
        sentiments_des = []
        start = datetime.now()
        for i, row in dataset[['productName', 'description']].iterrows():
            if (i % 1000 == 0) & (i != 0):
                end = datetime.now()
                print('epochs: {}, time executing: {}'.format(int(i / 1000), end - start))
                start = end

            try:
                sentiments_prod.extend(self._extract_good_sents(row['productName']))
            except:
                sentiments_prod.append('')

            try:
                sentiments_des.extend(self._extract_good_sents(row['description']))
            except:
                sentiments_des.append('')
        sentiments = [list(set(sentiments_prod).union(set(sentiments_des)))]
        dataset['sentiments'] = sentiments
        return dataset


    # I. tag 1: Kết hợp giữa cateName và description =======================================================================================================
    # Ví dụ: cateName = 'điện thoại', description = 'màu xanh' ==> 'điện thoại màu xanh'
    def _gen_tag1(self, cate_sent):
        cate_sent_tags = []
        cateName = cate_sent['cateName'].lower()
        for sent in cate_sent['sentiments']:
            if sent not in cateName:
                cate_sent_tags.append(' '.join([cateName, sent]))
        return cate_sent_tags


    # II. tag 2: Tên sản phẩm chính + description ==========================================================================================================
    # Ví dụ: cateName = 'điện thoại smartphone', description = 'màu xanh' ==> 'điện thoại smartphone màu xanh'
    def _gen_tag2(self, dataset):
        dataset['productNameClean'] = [self._clean_text(item) for item in dataset['productName']]
        dataset['productNameClean'] = [self._clean_text(item) for item in dataset['productName']]
        lm_tokenizer = dict_models.LongMatchingTokenizer()
        tok_core_nlp = [lm_tokenizer.tokenize(item) for item in dataset['productNameClean']]
        dataset['tok_core_nlp'] = tok_core_nlp
        # pos tag tên sản phẩm
        # mô tả các nhãn post: https://github.com/undertheseanlp/underthesea/wiki/M%C3%B4-t%E1%BA%A3-d%E1%BB%AF-li%E1%BB%87u-b%C3%A0i-to%C3%A1n-POS-Tag
        dataset['pos_tag'] = [pos_tag(item) for item in dataset['productNameClean']]
        # Tìm danh từ đầu trong tag
        dataset['min_pos_tag_N'] = [self._find_position_N(item) for item in dataset['pos_tag']]
        # Xác định tên sản phẩm
        dataset['main_product_N'] = [self._main_product_N(item['tok_core_nlp'], item['min_pos_tag_N']) for k, item in
                                     dataset[['tok_core_nlp', 'min_pos_tag_N']].iterrows()]
        # Xác định vị trí tên sản phẩm chính
        dataset['len_main_product_N'] = [len(item.split()) for item in dataset['main_product_N']]
        # Danh sách các danh từ đặc biệt là những từ có độ dài 1.
        self.special_N = set(dataset[dataset['len_main_product_N'] == 1]['main_product_N'])
        dataset['min_pos_tag_remove_short_N'] = [self._find_position_remove_short_N(item)
                                                 for item in dataset['pos_tag']]
        dataset['main_product_remove_short_N'] = [self._main_product_N(item['tok_core_nlp'], item['min_pos_tag_remove_short_N'])
                                                  for k, item in dataset[['tok_core_nlp', 'min_pos_tag_remove_short_N']].iterrows()]
        dataset['tag2'] = [self._gen_tag_main_product_desc(item)
                           for k, item in dataset[['main_product_remove_short_N', 'sentiments']].iterrows()]
        return dataset

    # Tìm vị trí đầu tiên mà từ bắt đầu là danh từ trong thuật toán pos_tag
    def _find_position_N(self, pos_tag):
        pos = 0
        for item in enumerate(pos_tag):
            # nếu là danh từ đầu tiên thì trả về pos
            if ('N' in item[1]):
                return pos
            else:
                pos += len(str(item[0]).split())
        return pos


    # Xuất phát từ danh từ trọng tâm (tên sản phẩm) thường đứng đầu nên ta sẽ ghép các từ từ tok_core_nlp tại vị trí từ min_pos_tag_N đổ về trước
    def _main_product_N(self, tokenization, min_pos_tag_N=0):
        pos = 0
        for i, item in enumerate(tokenization):
            if pos > min_pos_tag_N:
                return re.sub('_', ' ', ' '.join(tokenization[:i]))
            else:
                pos += len(item.split('_'))
        return re.sub('_', ' ', ' '.join(tokenization[:1]))

    # Loại bỏ những main product chỉ có độ dài là 1.
    def _find_position_remove_short_N(self, pos_tag):
        pos = 0
        for i, item in enumerate(pos_tag):
            # Nếu từ đầu tiên là danh từ và nằm trong danh sách special_N chọn từ tiếp theo (tiếp tục vòng lặp)
            if (i == 0) & ('N' in item[1]) & (item[0] in self.special_N):
                next
            # Nếu từ tiếp theo là danh từ trả về kết quả
            elif ('N' in item[1]):
                pos += len(item[0].split())
                return pos
            # Nếu không thì cộng vào độ dài của các từ liền trước
            else:
                pos += len(item[0].split())
        return pos

    # Tạo tag 2: Kết hợp giữa main_product_remove_short_N và description
    def _gen_tag_main_product_desc(self, main_product_sent):
        main_product_sent_tags = []
        for sent in main_product_sent['sentiments']:
            if sent not in main_product_sent['main_product_remove_short_N']:
                main_product_sent_tags.append(' '.join([main_product_sent['main_product_remove_short_N'], sent]))
        return main_product_sent_tags

    # III. tag 3: kết hợp giữa cateName và BranchName ==========================================================================================================
    # Ví dụ: cateName = 'điện thoại', branchName = 'samsung'
    # cateName + branchName = 'điện thoại samsung'
    def _gen_tag3(self, dataset):
        tag3 = []
        for k, (cateName, branch_name) in dataset[['cateName', 'branchName']].iterrows():
            branch_name = str(branch_name).lower()
            cateName = cateName.lower()
            if branch_name in cateName:
                tag3.append(cateName)
            elif cateName == '':
                tag3.append('')
            else:
                # Loại bỏ branch name nếu như đã xuất hiện trong cateName.
                tag = ' '.join([cateName, branch_name]).split()
                tag = ' '.join(sorted(set(tag), key=tag.index))
                tag3.append(tag)
        return tag3

    # IV. tag 4: tên sản phẩm chính + branchName =================================================================================================
    # Ví dụ: tên sản phẩm chính = 'điện thoại smartphone' + branchName = 'samsung' ==> 'điện thoại smartphone samsung'
    def _gen_tag4(self, dataset):
        tag4 = []
        for k, (main_product, branch_name) in dataset[['main_product_remove_short_N', 'branchName']].iterrows():
            branch_name = str(branch_name).lower()
            if branch_name in main_product:
                tag4.append(main_product)
            else:
                # remove duplicate word
                tag = ' '.join([main_product, branch_name]).split()
                tag = ' '.join(sorted(set(tag), key=tag.index))
                tag4.append(tag)
        return tag4

    # Kết hợp toàn bộ các tag1 - tag4
    def _combine_tags(self, dataset):
        keyword_suggestions = []
        for k, row in dataset[['tag2', 'tag3', 'tag4']].iterrows():
            row_kw = row['tag2']
            row_kw += [row['tag3']]
            row_kw += [row['tag4']]
            keyword_suggestions.append(row_kw)
        keyword_suggestions = [set(item) for item in keyword_suggestions]
        dataset['tags'] = [[item for item in tags] for tags in keyword_suggestions]
        return dataset


tagProduct = TagProductAll(args.link)
result = tagProduct._create_dataset()
result.to_csv(args.link_new)
result = result[['productCode', 'productName','tags']]
if os.path.exists(args.link_old):
    result_old = pd.read_csv(args.link_old, header = 0, index_col = 0)
    result = pd.concat([result_old, result], axis = 0)
    result.index = np.arange(result.shape[0])
    result.to_csv(args.link_output)
    result.to_csv(args.link_old)
else:
    result.to_csv(args.link_old)

config sentiments:  good_sentiments.json
config sentiments:  good_sentiments.json


AttributeError: 'dict' object has no attribute 'link_cate'

In [287]:
args['link_cate']

'FullDisplayCate.xlsx'

In [85]:
def _gen_dict_desc(text):
    desc_dict = defaultdict()
    for k, item in enumerate(text.split()):
        if item in desc_dict:
            desc_dict[item].append(k)
        else:
            desc_dict[item] = [k]
    return desc_dict

# Tính toán khoảng cách nhỏ nhất của 2 cụm vị trí
# Ví dụ: 'trời xanh mây xanh' sau khi tạo từ điển vị trí sẽ là {'trời':[0], 'xanh':[1, 3], 'mây': [2]}
# Đo khoảng cách giữa từ 'xanh' và 'trời' sẽ là 1. Mục đích là để tìm các từ có thể ghép cặp với nhau trong 1 khoảng cách nhất định nếu việc tìm từ đó liền kề nhau là không thể.
# VD: 'trong và sạch' có thể được ghép cặp coi như từ 'trong sạch' bằng cách so khớp khoảng cách.
def _cross_distance(first_list, second_list):
    return np.min([(y - x) for x in first_list for y in second_list if (y - x) >= 0])

def _check_collocation(first, last, desc_dict, min_distance=2):
    try:
        distance = _cross_distance(desc_dict[first], desc_dict[last])
    except:
        return False
    if (distance <= min_distance) & (distance >= 0):
        return True
    else:
        return False

# print(_check_collocation('cho', 'tươi', desc_dict = desc_dict))

# Loại bỏ các kí tự đặc biệt bằng khoảng trắng
def _clean_text(text, regex=r'[-\(\)\"#/@;:<>{}\[\]`+=~%$&|.*!?,|\u200b]'):
    text = re.sub(regex, r' ', text.lower())
    text = re.sub(r'\s+', r' ', text)
    return text

# Kiểm tra một cụm từ có trong mô tả hay không với khoảng cách là 2
def _check_desc(search, description, min_distance=2):
    words = search.split()
    description = _clean_text(description)
    desc_dict = _gen_dict_desc(description)
    if len(words) == 1:
        if not words[0] in desc_dict:
            return False
    else:
        for i in range(len(words) - 1):
            if not _check_collocation(words[i], words[i + 1], desc_dict, min_distance):
                return False
    return True

In [268]:
_check_desc('bộ nhớ 128gb', "- Màn hình tràn viền Super AMOLED 6.3 18.5:9 FullHD - Bộ xử lý: Snapdragon 660 Octa-Core 2.2GHz - Hệ điều hành: Android 8.0 Oreo + Samsung Experience 9.0 - RAM:6GB - Bộ nhớ trong:128GB - Camera sau:24MP, 10MP, 8MP và 5MP (4 camera) - Camera trước:24MP - Pin: 3.800mAh Siêu tiết kiện pin - Thẻ nhớ:MicroSD, hỗ trợ tối đa 512GB")
_check_desc('128gb','Samsung Galaxy A9 (2018) 6GB / 128GB Hồng - Hàng chính hãng')

True

In [266]:
with open('good_sentiments.json', 'r', errors = 'ignore') as fn:
    sentiments= json.load(fn)
sentiments

['bền',
 'đẹp',
 'chắc chắn',
 'tốt',
 'ngon',
 'rẻ',
 'giá rẻ',
 'siêu rẻ',
 'trợ giá',
 'an toàn',
 'đặc biệt',
 'nhỏ',
 'nhỏ gọn',
 'ít ồn',
 'thoải mái',
 'vững chắc',
 'trẻ trung',
 'năng động',
 'sinh động',
 'độ rung thấp',
 'không rung lắc',
 'mạnh mẽ',
 'dễ điều khiển',
 'điều khiển',
 'ngộ nghĩnh',
 'nhiệt huyết',
 'đam mê',
 'lịch sự',
 'trang trọng',
 'xì teen',
 'nhã nhặn',
 'mềm mại',
 'suôn mượt',
 'đậm đặc',
 'tươi ngon',
 'đặc sắc',
 'đặc biệt',
 'hảo hạng',
 'thượng hạng',
 'đột phá',
 'màn hình vô cực',
 'vượt trội',
 'tiên tiến',
 'thông minh',
 'hiện đại',
 'camera kép',
 'pin bền bỉ',
 'sạc nhanh',
 'đa nhiệm',
 'giải trí',
 '32GB',
 '64GB',
 '128GB',
 'Bộ nhớ 32GB',
 'Bộ nhớ 64GB',
 'Bộ nhớ 128GB',
 'RAM 3GB',
 'RAM 2GB',
 'RAM 4GB',
 'RAM 6GB',
 'RAM 8GB',
 '3GB',
 '2GB',
 '4GB',
 '6GB',
 '8GB',
 '10000 mAh',
 '2000 mAh',
 '2700 mAh',
 '2870 mAh',
 '2942 mAh',
 '3000 mAh',
 '3070 mAh',
 '3100 mAh',
 '3120 mAh',
 '3200 mAh',
 '3210 mAh',
 '3260 mAh',
 '3300 mAh',

In [82]:
'thích hợp' in sentiments

False

In [281]:
good_sentiments = ['bền',
'đẹp',
'chắc chắn',
'tốt',
'ngon',
'rẻ',
'giá rẻ',
'siêu rẻ',
'trợ giá',
'an toàn',
'đặc biệt',
'nhỏ',
'nhỏ gọn',
'ít ồn',
'thoải mái',
'vững chắc',
'trẻ trung',
'năng động',
'sinh động',
'độ rung thấp',
'không rung lắc',
'mạnh mẽ',
'dễ điều khiển',
'điều khiển',
'ngộ nghĩnh',
'nhiệt huyết',
'đam mê',
'lịch sự',
'trang trọng',
'xì teen',
'nhã nhặn',
'mềm mại',
'suôn mượt',
'đậm đặc',
'tươi ngon',
'hương trà',
'hương trà xanh',
'đặc sắc',
'đặc biệt',
'hảo hạng',
'thượng hạng',
'đột phá',
'màn hình vô cực',
'vượt trội',
'tiên tiến',
'thông minh',
'hiện đại',
'camera kép',
'pin bền bỉ',
'sạc nhanh',
'đa nhiệm',
'giải trí',
'32GB',
'64GB',
'128GB',
'Bộ nhớ 32GB',
'Bộ nhớ 64GB',
'Bộ nhớ 128GB',
'RAM 3GB',
'RAM 2GB',
'RAM 4GB',
'RAM 6GB',
'RAM 8GB',
'3GB',
'2GB',
'4GB',
'6GB',
'8GB',
'10000 mAh',
'2000 mAh',
'2700 mAh',
'2870 mAh',
'2942 mAh',
'3000 mAh',
'3070 mAh',
'3100 mAh',
'3120 mAh',
'3200 mAh',
'3210 mAh',
'3260 mAh',
'3300 mAh',
'3320 mAh',
'3330 mAh',
'3350 mAh',
'3360 mAh',
'3400 mAh',
'3500 mAh',
'3540 mAh',
'3600 mAh',
'3650 mAh',
'3700 mAh',
'3750 mAh',
'3765 mAh',
'3800 mAh',
'3930 mAh',
'4000 mAh',
'4000 mAh,',
'4380 mAh',
'4500 mAh',
'5000 mAh',
'800 mAh',
'pin 10000 mAh',
'pin 2000 mAh',
'pin 2700 mAh',
'pin 2870 mAh',
'pin 2942 mAh',
'pin 3000 mAh',
'pin 3070 mAh',
'pin 3100 mAh',
'pin 3120 mAh',
'pin 3200 mAh',
'pin 3210 mAh',
'pin 3260 mAh',
'pin 3300 mAh',
'pin 3320 mAh',
'pin 3330 mAh',
'pin 3350 mAh',
'pin 3360 mAh',
'pin 3400 mAh',
'pin 3500 mAh',
'pin 3540 mAh',
'pin 3600 mAh',
'pin 3650 mAh',
'pin 3700 mAh',
'pin 3750 mAh',
'pin 3765 mAh',
'pin 3800 mAh',
'pin 3930 mAh',
'pin 4000 mAh',
'pin 4000 mAh,',
'pin 4380 mAh',
'pin 4500 mAh',
'pin 5000 mAh',
'pin 800 mAh',
'10 MP',
'12 MP',
'122 MP',
'123 MP',
'13 MP',
'131 MP',
'16 MP',
'18 MP',
'19 MP',
'2 MP',
'20 MP',
'207 MP',
'21 MP',
'22 MP',
'23 MP',
'24 MP',
'248 MP',
'25 MP',
'3 MP',
'30 MP',
'31 MP',
'315MP',
'32 MP',
'37 MP',
'4 MP',
'40 MP',
'411 MP',
'48 MP',
'5 MP',
'50 MP',
'51 MP',
'7 MP',
'8 MP',
'80 MP',
'87 MP',
'9 MP',
'camera 10 MP',
'camera 12 MP',
'camera 122 MP',
'camera 123 MP',
'camera 13 MP',
'camera 131 MP',
'camera 16 MP',
'camera 18 MP',
'camera 19 MP',
'camera 2 MP',
'camera 20 MP',
'camera 207 MP',
'camera 21 MP',
'camera 22 MP',
'camera 23 MP',
'camera 24 MP',
'camera 248 MP',
'camera 25 MP',
'camera 3 MP',
'camera 30 MP',
'camera 31 MP',
'camera 315MP',
'camera 32 MP',
'camera 37 MP',
'camera 4 MP',
'camera 40 MP',
'camera 411 MP',
'camera 48 MP',
'camera 5 MP',
'camera 50 MP',
'camera 51 MP',
'camera 7 MP',
'camera 8 MP',
'camera 80 MP',
'camera 87 MP',
'camera 9 MP',
'camera siêu rộng',
'camera kép',
'màn hình tràn viền',
'cảm ứng đa nhiệm',
'cho người già',
'cho bé',
'trẻ sơ sinh',
'sơ sinh',
'pin khủng',
'sống động',
'hình ảnh trung thực',
'4k ultra HD',
'ultra HD',
'oled',
'55 inch',
'49 inch',
'10 inch',
'14 inch',
'13 inch',
'17 inch',
'21 inch',
'32 inch',
'25 inch',
'65 inch',
'43 inch',
'40 inch',
'5W',
'10W',
'15W',
'20W',
'25W',
'30W',
'35W',
'40W',
'45W',
'50W',
'55W',
'60W',
'65W',
'70W',
'75W',
'80W',
'85W',
'90W',
'95W',
'100W',
'105W',
'110W',
'115W',
'120W',
'125W',
'130W',
'135W',
'140W',
'145W',
'150W',
'200W',
'300W',
'400W',
'500W',
'600W',
'700W',
'800W',
'900W',
'1000W',
'1100W',
'1200W',
'1300W',
'1400W',
'1500W',
'1600W',
'220V',
'110V',
'smart',
'màu sắc phong phú',
'phân giải cao',
'trợ lý google',
'internet',
'full hd',
'android',
'màn hình cong',
'màn hình phẳng',
'vành đĩa',
'bản cao cấp',
'phân phối lớn',
'cho trẻ em',
'cho người lớn',
'cho gia đình',
'cho 2 người',
'cho cặp đôi',
'cho 4 người',
'hàng cũ',
'đời mới',
'nan hoa',
'điện vin',
'gaming',
'chơi game',
'cũ',
'siêu bền',
'nhôm nguyên khối',
'core 13',
'core i5',
'core i7',
'chống rung',
'giá rẻ',
'bàn không dây',
'đuôi khỉ',
'cục gạch',
'cố định',
'để bàn',
'cảm ứng',
'tách đường',
'cao tần',
'nắp rời',
'nắp gài',
'1 lít',
'1l',
'0.6 lít',
'0.6 l',
'0.7 lít',
'0.7 l',
'1.8 lít',
'1.8 l',
'chống dính',
'không dầu',
'đáy từ',
'nồi hấp',
'áp suất',
'không gỉ',
'nhỏ gọn',
'tiện dụng',
'chống rỉ sét',
'chống ồn',
'28 giờ',
'âm thanh chất lượng',
'âm thanh chân thực',
'tích hợp NFC',
'chống nắng',
'đi biển',
'2 dây',
'cách tân',
'vải Cotton',
'cổ bẻ',
'lịch lãm',
'hình tròn',
'đính đá',
'tinh xảo',
'màu xanh',
'màu xám',
'màu xám đen',
'màu đỏ',
'màu đỏ san hô',
'màu tím',
'màu hồng',
'màu đen',
'màu đen bóng',
'màu bạc',
'màu đồng',
'màu trắng',
'màu nâu',
'màu vàng',
'màu vàng kim',
'màu vàng hồng',
'màu vàng đồng',                   
'màu bạc',
'màu xanh dương',
'màu xanh nước biển',
'màu xanh lá cây',
'xanh',
'xám',
'xám đen',
'đỏ',
'đỏ san hô',
'tím',
'hồng',
'đen',
'đen bóng',
'bạc',
'đồng',
'trắng',
'nâu',
'vàng',
'vàng kim',
'vàng hồng',
'vàng đồng',
'bạc',
'xanh dương',
'xanh nước biển',
'xanh lá cây',
'rộng',
'bó sát',
'tôn dáng',
'lợi sữa',
'dinh dưỡng',
'nguyên chất',
'giảm cân',
'làm đẹp',
'trị viêm loét',
'tươi',
'chống ung thư',
'đẹp da',
'trắng da',
'sát khuẩn',
'làm sạch vết thương',
'chống viêm',
'kháng khuẩn',
'hạ nhiệt',
'giảm sốt',
'trị sổ mũi',
'trị cảm cúm',
'trị đau bụng',
'trị đau đầu',
'trị thần kinh toại',
'trị ho',
'kích thích tiêu hóa',
'long đờm',
'sắc nét',
'sống động',
'vui vẻ',
'vé vui chơi',
'mộc mạc',
'hài hòa',
'cảm xúc',
'chân thành',
'nhân ái',
'tươi mát',
'tinh tế',
'tỉ mỉ',
'thép không gỉ',
'nữ tính',
'chất liệu nhựa',
'co dãn',
'đàn hồi',
'nhẹ nhàng, cảm xúc',
'tay dài',
'không độc hại',
'hoạt hình',
'không gây ngứa',
'không rụng lông',
'mặt trơn',
'tay ngắn',
'dị thường',
'kì quái',
'gần gũi',
'chân thật',
'cài nút',
'lai bầu',
'chuẩn phom',
'vải kaki',
'cạnh đính đỉa',
'kẻ caro',
'thoáng mát',
'cảm biến',
'mát lạnh',
'tiết kiệm không gian',
'độ bền cao',
'hiện đại',
'tiện lợi',
'2 chiều',
'sạc nhanh',
'hoàn hảo',
'nhựa bền',
'nhựa',
'thoáng mát',
'thấm hút',
'thân thiện da',
'họa tiết sọc',
'cúp chéo',
'hẹn giờ',
'tự động',
'tản nhiệt đều',
'thú vị',
'ngắn gọn',
'dễ tháo lắp',
'độc đáo',
'chất lượng',
'tươi mới',
'khó phai',
'mềm mại',
'thoát ẩm',
'lọc Nano silver',
'van điện từ',
'đánh giá kỹ năng',
'đánh giá kiến thức',
'hướng dẫn',
'trung thực',
'không nháy',
'linh hoạt',
'xoay',
'gập',
'ôn luyện',
'bám sát',
'tham khảo',
'ngừa lão hóa',
'căng da',
'da căng',
'săn chắc',
'trung tính',
'tiện ích',
'2 đầu',
'co dãn',
'dẻo',
'size l',
'size xxl',
'size m',
'size xl',
'size nb',
'size s',
'size n',
'size xs']

In [282]:
def _save_json(filename, obj):
    with open(filename, "w") as fp:
        json.dump(obj, fp)
        
_save_json('good_sentiments.json', good_sentiments)    
good_sentiments

['bền',
 'đẹp',
 'chắc chắn',
 'tốt',
 'ngon',
 'rẻ',
 'giá rẻ',
 'siêu rẻ',
 'trợ giá',
 'an toàn',
 'đặc biệt',
 'nhỏ',
 'nhỏ gọn',
 'ít ồn',
 'thoải mái',
 'vững chắc',
 'trẻ trung',
 'năng động',
 'sinh động',
 'độ rung thấp',
 'không rung lắc',
 'mạnh mẽ',
 'dễ điều khiển',
 'điều khiển',
 'ngộ nghĩnh',
 'nhiệt huyết',
 'đam mê',
 'lịch sự',
 'trang trọng',
 'xì teen',
 'nhã nhặn',
 'mềm mại',
 'suôn mượt',
 'đậm đặc',
 'tươi ngon',
 'hương trà',
 'hương trà xanh',
 'đặc sắc',
 'đặc biệt',
 'hảo hạng',
 'thượng hạng',
 'đột phá',
 'màn hình vô cực',
 'vượt trội',
 'tiên tiến',
 'thông minh',
 'hiện đại',
 'camera kép',
 'pin bền bỉ',
 'sạc nhanh',
 'đa nhiệm',
 'giải trí',
 '32GB',
 '64GB',
 '128GB',
 'Bộ nhớ 32GB',
 'Bộ nhớ 64GB',
 'Bộ nhớ 128GB',
 'RAM 3GB',
 'RAM 2GB',
 'RAM 4GB',
 'RAM 6GB',
 'RAM 8GB',
 '3GB',
 '2GB',
 '4GB',
 '6GB',
 '8GB',
 '10000 mAh',
 '2000 mAh',
 '2700 mAh',
 '2870 mAh',
 '2942 mAh',
 '3000 mAh',
 '3070 mAh',
 '3100 mAh',
 '3120 mAh',
 '3200 mAh',
 '321

In [121]:
# Đọc các model điện thoại
# import requests
# response = requests.get('https://en.wikipedia.org/wiki/Comparison_of_smartphones#2019')
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(response.content, 'html.parser')

In [129]:
# pd.read_html(soup.find('table'))
data = pd.read_html('https://en.wikipedia.org/wiki/Comparison_of_smartphones#2019')

In [151]:
battery = []
for i in range(12):
    bat = [element for element in list(data[i].iloc[:, 11]) if (len(str(element)) <= 10) & (str(element) != 'nan')]
    battery.extend(bat)
    
battery = set(battery)
battery

{'10000 mAh',
 '2,700 mAh',
 '2000 mAh',
 '2870 mAh',
 '2942 mAh',
 '3 MP AF',
 '3,040 mAh',
 '3,300 mAh',
 '3,520 mAh',
 '3.0 MP AF',
 '3.2\xa0MP',
 '3000 mAh',
 '3070 mAh',
 '3100 mAh',
 '3120 mAh',
 '3200 mAh',
 '3210 mAh',
 '3260 mAh',
 '3300 mAh',
 '3320 mAh',
 '3330 mAh',
 '3350 mAh',
 '3360mAh',
 '3400 mAh',
 '3400mAh',
 '3500 mAh',
 '3500mAh',
 '3540 mAh',
 '3600 mAh',
 '3650 mAh',
 '3700 mAh',
 '3750 mAh',
 '3765 mAh',
 '3800 mAh',
 '3930 mAh',
 '4000 mAh',
 '4000 mAh,',
 '4380 mAh',
 '4500 mAh',
 '5 MP',
 '5.0 MP AF',
 '5000 mAh',
 '800 mAh',
 'Battery',
 'Camera',
 'Display',
 'Other',
 'microSDHC'}

In [258]:
import re

def _filter_MP(text):
    try:
        x = ''.join(re.findall('[\d MP]', text))
        x = x.strip()
        x = [item.strip() for item in x.split('  ')]
    except:
        return ''
    return x

# _filter_MP('Triple 48 MP + 13 MP + 8 MP (rear camera), 16 MP (front camera)')

def _get_all_MP(dataset):
    MP_types = list(dataset.apply(lambda x: _filter_MP(x)))
    MP_types = list(set([item for item_list in MP_types for item in item_list]))
    return MP_types

MP0 = _get_all_MP(data[0].iloc[:, 14])
MP1 = _get_all_MP(data[1].iloc[:, 14])
MP2 = _get_all_MP(data[2].iloc[:, 13])
MP3 = _get_all_MP(data[3].iloc[:, 13])
MP4 = _get_all_MP(data[4].iloc[:, 13])
MP5 = _get_all_MP(data[5].iloc[:, 13])
MP6 = _get_all_MP(data[6].iloc[:, 11])
MP7 = _get_all_MP(data[7].iloc[:, 11])
MP8 = _get_all_MP(data[8].iloc[:, 11])
MP9 = _get_all_MP(data[9].iloc[:, 11])
MP10 = _get_all_MP(data[10].iloc[:, 11])

MPS = []
for i in range(10):
    mp_name = 'MP'+str(i)
    MP = eval(mp_name)
    MPS.extend(MP)
MPS = set(MPS)

In [259]:
MPS

{'',
 '03 MP',
 '031 MP',
 '03MP 640480',
 '10 MP',
 '1080',
 '108030',
 '12 MP',
 '12 MP 16',
 '12 MP 28',
 '120',
 '121 1080',
 '122 MP',
 '123 MP',
 '123MP',
 '1280',
 '13 MP',
 '13 MP 720',
 '131 MP',
 '13M',
 '13MP',
 '15',
 '16 MP',
 '18MP',
 '19 MP',
 '1920',
 '19201080',
 '19MP',
 '2 2 MP',
 '2 MP',
 '20',
 '20 MP',
 '20 MP 16',
 '207 MP',
 '21',
 '21 MP',
 '22',
 '22 26',
 '22 MP',
 '23 MP',
 '24',
 '24 720',
 '24 MP',
 '240',
 '2448',
 '248 MP',
 '25 MP',
 '25MP',
 '28',
 '2MP',
 '3',
 '30',
 '30 03 MP',
 '30 13 MP',
 '30 16 MP',
 '30 2 MP',
 '30 MP',
 '30 P',
 '315MP',
 '32 MP',
 '32MP',
 '37 MP',
 '3MP',
 '4',
 '4 MP',
 '4 MP 20',
 '4 MP P',
 '40 MP',
 '411 MP',
 '42160',
 '432',
 '48 MP',
 '480',
 '48030',
 '5 MP',
 '50 MP',
 '50MP',
 '51 MP',
 '5MP',
 '5MP 25601920',
 '6',
 '60',
 '60 P',
 '640480',
 '7 MP',
 '720',
 '72030',
 '720480',
 '768',
 '8',
 '8 MP',
 '8 MP 20',
 '8 MP 22',
 '8 MP 24',
 '8 MP 3264',
 '8 MP 32642448',
 '80 MP',
 '80MP',
 '87MP',
 '8MP',
 '8MP 3264

In [261]:
MPS = {
 '3 MP',
 '31 MP',
 '10 MP',
 '12 MP',
 '122 MP',
 '123 MP',
 '13 MP',
 '13 MP',
 '131 MP',
 '13 MP',
 '16 MP',
 '18 MP',
 '19 MP',
 '2 MP',
 '20 MP',
 '207 MP',
 '21 MP',
 '22 MP',
 '23 MP',
 '24 MP',
 '248 MP',
 '25 MP',
 '30 MP',
 '315MP',
 '32 MP',
 '37 MP',
 '4 MP',
 '40 MP',
 '411 MP',
 '48 MP',
 '5 MP',
 '50 MP',
 '51 MP',
 '7 MP',
 '8 MP',
 '80 MP',
 '87 MP',
 '9 MP'}
set(['camera ' + item for item in MPS])

{'camera 10 MP',
 'camera 12 MP',
 'camera 122 MP',
 'camera 123 MP',
 'camera 13 MP',
 'camera 131 MP',
 'camera 16 MP',
 'camera 18 MP',
 'camera 19 MP',
 'camera 2 MP',
 'camera 20 MP',
 'camera 207 MP',
 'camera 21 MP',
 'camera 22 MP',
 'camera 23 MP',
 'camera 24 MP',
 'camera 248 MP',
 'camera 25 MP',
 'camera 3 MP',
 'camera 30 MP',
 'camera 31 MP',
 'camera 315MP',
 'camera 32 MP',
 'camera 37 MP',
 'camera 4 MP',
 'camera 40 MP',
 'camera 411 MP',
 'camera 48 MP',
 'camera 5 MP',
 'camera 50 MP',
 'camera 51 MP',
 'camera 7 MP',
 'camera 8 MP',
 'camera 80 MP',
 'camera 87 MP',
 'camera 9 MP'}

In [270]:
x = ['hồng', '6GB', '128GB']
y = ['Bộ nhớ 128GB', '6GB', '128GB', 'RAM 6GB', 'màn hình tràn viền', 'android']

set(x).union(set(y))

{'128GB',
 '6GB',
 'Bộ nhớ 128GB',
 'RAM 6GB',
 'android',
 'hồng',
 'màn hình tràn viền'}